In [23]:
import laspy
import numpy as np
from scipy.interpolate import griddata
import rasterio
import pyproj
import cv2

# Read in the LAS file using laspy

las_file2 = laspy.read('USGS_LPC_IL_HicksDome_FluorsparDistrict_2019_D19_2339_5650.laz')
# Set the CRS information to projection
las_file2.header.scale[0] = 0.01
las_file2.header.scale[1] = 0.01
las_file2.header.scale[2] = 0.001
las_file2.write("updated_file.las")
las_file2 = laspy.read('updated_file.las')



# Define the CRS for the output raster
print('Enter the epsg for your laz file:')
crs = pyproj.CRS.from_epsg(input())  

# Extract the x, y, and z coordinates from the LAS file
x = las_file2.x
y = las_file2.y
z = las_file2.z

# Define the resolution of the DEM
resolution = 1

# Calculate the grid bounds based on the x and y coordinates
x_min = np.floor(min(x))
x_max = np.ceil(max(x))
y_min = np.floor(min(y))
y_max = np.ceil(max(y))

# Generate the grid of points for the DEM
grid_x, grid_y = np.meshgrid(np.arange(x_min, x_max , resolution), 
                             np.arange(y_min, y_max , resolution))

# Generate the DEM using linear interpolation of the point cloud
dem = griddata((x, y), z, (grid_x, grid_y), method='nearest')


# Save the DEM to a GeoTIFF file using rasterio
with rasterio.open(
        'dsm.tif',
        'w',
        driver='GTiff',
        height=dem.shape[0],
        width=dem.shape[1],
        count=1,
        dtype=dem.dtype,
        crs=crs,
        transform=rasterio.transform.Affine(resolution, 0, x_min, 0, resolution, y_min)) as dst:
    dst.write(dem, 1)
print('Success')

Enter the epsg for your laz file:


 8734


sucsess


In [24]:
from rasterio.transform import from_origin
from scipy.interpolate import griddata
import scipy


# Load LiDAR data
las_file = laspy.read('updated_file.las')
points = np.vstack((las_file.x, las_file.y, las_file.z)).T

# Classify ground points
ground_points = points[las_file.classification == 2]

# Define the resolution of the output DTM
resolution = 1

# Determine the bounds of the point cloud
min_x, max_x = np.min(points[:,0]), np.max(points[:,0])
min_y, max_y = np.min(points[:,1]), np.max(points[:,1])

# Calculate the size of the output raster
width = int(np.ceil((max_x - min_x) / resolution))
height = int(np.ceil((max_y - min_y) / resolution))

# Create the output raster profile
profile = {
    'driver': 'GTiff',
    'height': height,
    'width': width,
    'count': 1,
    'dtype': 'float32',
    'crs': crs,
    'transform': from_origin(min_x, min_y, resolution, -resolution)
}

# Create an empty numpy array for the output DTM
dtm = np.zeros((height, width), dtype=np.float32)
counts = np.zeros((height, width), dtype=np.int32)

# Create a KDTree from the x, y coordinates of the ground points
tree = scipy.spatial.cKDTree(ground_points[:, :2])

# Create a mesh grid for the output raster
mesh_x, mesh_y = np.meshgrid(np.arange(min_x, max_x, resolution), np.arange(min_y, max_y, resolution))

# Interpolate the z values of the ground points onto the mesh grid
values = tree.query(np.vstack((mesh_x.ravel(), mesh_y.ravel())).T)[0]
mesh_z = values.reshape(mesh_x.shape)


# Load the DSM
dsm_file = laspy.read('updated_file.las')
dsm_points = np.vstack((dsm_file.x, dsm_file.y, dsm_file.z)).T

# Classify points as aboveground features (non-ground points)
non_ground_points = dsm_points[dsm_file.classification != 2]

# Interpolate the non-ground points onto the mesh grid
non_ground_z = griddata(non_ground_points[:, :2], non_ground_points[:, 2], (mesh_x, mesh_y), method='linear')

# Subtract the interpolated non-ground values from the interpolated ground values
dtm = mesh_z - non_ground_z

# Write the output raster to a file
with rasterio.open('dtm.tif', 'w', **profile) as dst:
    dst.write(dtm, 1)

In [25]:
# Load DSM and DTM
with rasterio.open('dsm.tif') as src:
    dsm = src.read(1)
    dsm_meta = src.profile

with rasterio.open('dtm.tif') as src:
    dtm = src.read(1)

# Compute NDHM
ndhm = dsm - dtm

# Write NDHM to file
ndhm_meta = dsm_meta.copy()
ndhm_meta['dtype'] = 'float32'
with rasterio.open('ndhm.tif', 'w', **ndhm_meta) as dst:
    dst.write(ndhm.astype(np.float32), 1)

In [28]:
from rasterio.plot import show
from rasterio.mask import mask
import cv2

# Open the NDHM image
with rasterio.open('ndhm.tif') as src:
    ndhm_img = src.read(1)
    profile = src.profile.copy()

# Threshold the NDHM image to separate the buildings
# Convert the NDHM image to 8-bit
ndhm_img_8bit = cv2.normalize(ndhm_img, None, 0, 255, cv2.NORM_MINMAX, cv2.CV_8UC1)

# Apply adaptive thresholding to separate the buildings
ndhm_img_thresh = cv2.adaptiveThreshold(ndhm_img_8bit, 255, cv2.ADAPTIVE_THRESH_GAUSSIAN_C, cv2.THRESH_BINARY_INV, 51, 13.3)

# Find the contours of the buildings
contours, _ = cv2.findContours(ndhm_img_thresh.astype(np.uint8), cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)

# Create a binary mask for the buildings
building_mask = np.zeros_like(ndhm_img_thresh, dtype=np.uint8)

# # Set a minimum building size ***(doest work properly yet)
min_size = 160

# Compute the pixel size of a square meter ***(doest work properly yet)
pixel_size = abs(profile['transform'][0])

# Filter out contours with low squareness or small size (likely vegetation or noise)
for contour in contours:
    # Compute the squareness and size of the contour
    rect = cv2.minAreaRect(contour)
    w, h = rect[1]
    if w < h:
        w, h = h, w
    squareness = w / h if h != 0 else 0
    size = w * h * pixel_size ** 2

    # Filter out contours with low squareness or small size (likely vegetation or noise)
    if squareness >= 0.9 and size >= min_size:
        cv2.drawContours(building_mask, [contour], -1, 255, -1)

# Update the metadata of the output file to reflect a binary output
profile.update(count=1, dtype=rasterio.uint8)

# Write the new GeoTIFF file with the binary mask of the buildings
with rasterio.open('buildings.tif', 'w', **profile) as dst:
    dst.write(building_mask.astype(rasterio.uint8), 1)
print('All of our steps are done.')